# Dependencies

In [ ]:
!pip install numpy
!pip install pandas
!pip install nltk


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
^C
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.9/301.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!sudo apt install -y --no-install-recommends g++ protobuf-compiler libprotobuf-dev
!pip install gcld3

# Imports

In [1]:
import gcld3
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import string
import re
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

# Constants

In [7]:
COLUMNS = ['tweet', 'likes', 'retweet_count', 'user_screen_name', 'user_description', 'user_followers_count']
LANG = 'en'
TW_USERNAME_REGEX = r"@[a-zA-Z0-9_]{0,15}"
URL_REGEX = r"\b(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][" \
            r"a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2," \
            r"}|www\.[a-zA-Z0-9]+\.[^\s]{2,})\b"
SPACES_REGEX = r"\s+"

In [ ]:
from google.colab import drive
GOOGLE_DRIVE_BASE_DIR = "/content/drive/MyDrive/ITBA/Quinto Año/Segundo Cuatrimestre/NLP/TP"
drive.mount("/content/drive")

# Helper functions

In [ ]:
def get_selected_columns(df, columns):
    return df[columns]


def delete_hashtag_symbol(df):
    df['tweet'] = df['tweet'].replace('#', '', regex=True)
    return df


def delete_twitter_username(df):
    df['tweet'] = df['tweet'].replace(TW_USERNAME_REGEX, '', regex=True)
    return df


def delete_urls(df):
    df['tweet'] = df['tweet'].replace(URL_REGEX, '', regex=True)
    return df


def delete_multiple_spaces(df):
    df['tweet'] = df['tweet'].replace(SPACES_REGEX, '', regex=True)


def is_lang(row, detector, lang='en'):
    prediction = detector.FindLanguage(text=row['tweet'])
    if prediction.language == lang and prediction.is_reliable:
        return True
    else:
        return False


def filter_by_language(df, lang='en'):
    detector = gcld3.NNetLanguageIdentifier(min_num_bytes=50, max_num_bytes=2048)

    mask = df.apply(is_lang, axis=1, detector=detector, lang=lang)
    return df[mask]


def is_feeling(row, sia, feeling, threshold):
    sentiment_scores = sia.polarity_scores(row['tweet'])
    if sentiment_scores[feeling] > threshold:
        return True
    else:
        return False


def filter_by_sentiment(df, feeling, threshold=0.4):
    sia = SentimentIntensityAnalyzer()

    mask = df.apply(is_feeling, axis=1, sia=sia, feeling=feeling, threshold=threshold)
    return df[mask]


def is_relevant(row, min_likes, min_retweets):
    try:
        likes = float(row['likes'])
        retweets = float(row['retweet_count'])
        if likes > min_likes or retweets > min_retweets:
            return True
        else:
            return False
    except:
        return False


def filter_by_relevance(df, min_likes, min_retweets):
    mask = df.apply(is_relevant, axis=1, min_likes=min_likes, min_retweets=min_retweets)
    return df[mask]

# Tweets filtering and processing

In [ ]:
trump_df = pd.read_csv(f"{GOOGLE_DRIVE_BASE_DIR}/hashtag_donaldtrump.csv", sep=',', lineterminator='\n',
                       parse_dates=True, low_memory=False)
biden_df = pd.read_csv(f"{GOOGLE_DRIVE_BASE_DIR}/hashtag_joebiden.csv", sep=',', lineterminator='\n',
                       parse_dates=True, low_memory=False)

trump_tweets_count = len(trump_df)
biden_tweets_count = len(biden_df)

print(f'Total Trump Tweets: {trump_tweets_count}')
print(f'Total Biden Tweets: {biden_tweets_count}')

# Filtro de campos de interes
trump_df = get_selected_columns(trump_df, COLUMNS)
biden_df = get_selected_columns(biden_df, COLUMNS)

# Filtro por longitud
trump_df = trump_df[trump_df['tweet'].str.len() >= 50]
biden_df = biden_df[biden_df['tweet'].str.len() >= 50]

# Filtro por contenido
# Links
trump_df['tweet'] = trump_df['tweet'].replace(URL_REGEX, '', regex=True)
biden_df['tweet'] = biden_df['tweet'].replace(URL_REGEX, '', regex=True)
# Arrobas de respuesta o mencion
trump_df['tweet'] = trump_df['tweet'].replace(TW_USERNAME_REGEX, '', regex=True)
biden_df['tweet'] = biden_df['tweet'].replace(TW_USERNAME_REGEX, '', regex=True)
# Espacios en blanco de mas
trump_df['tweet'] = trump_df['tweet'].replace(SPACES_REGEX, ' ', regex=True)
biden_df['tweet'] = biden_df['tweet'].replace(SPACES_REGEX, ' ', regex=True)
# Caracteres inválidos
trump_df['tweet'] = trump_df['tweet'].replace("&amp;", '', regex=True)
biden_df['tweet'] = biden_df['tweet'].replace("&amp;", '', regex=True)
# Simbolo de hashtag
trump_df['tweet'] = trump_df['tweet'].replace("#", '', regex=True)
biden_df['tweet'] = biden_df['tweet'].replace("#", '', regex=True)
# Filtro por relevancia
print(f'\nTrump tweets mean')
print(trump_df[['likes', 'retweet_count']].mean())
print(f'\nTrump tweets max')
print(trump_df[['likes', 'retweet_count']].max())

print(f'\nBiden tweets mean')
print(biden_df[['likes', 'retweet_count']].mean())
print(f'\nBiden tweets max')
print(biden_df[['likes', 'retweet_count']].max())

trump_df = filter_by_relevance(trump_df, min_likes=10, min_retweets=10)
biden_df = filter_by_relevance(biden_df, min_likes=10, min_retweets=10)

print(f"Filtered trump tweets: {len(trump_df)}")
print(f"Filtered biden tweets: {len(biden_df)}")

# Lenguage Ingles
trump_df = filter_by_language(trump_df, lang=LANG)
biden_df = filter_by_language(biden_df, lang=LANG)

print(f"Filtered trump tweets: {len(trump_df)}")
print(f"Filtered biden tweets: {len(biden_df)}")

In [ ]:
# Filtro por sentimiento
trump_df = filter_by_sentiment(trump_df, feeling='neg', threshold=0.3)
biden_df = filter_by_sentiment(biden_df, feeling='neg', threshold=0.3)

print(f"Filtered trump tweets: {len(trump_df)}")
print(f"Filtered biden tweets: {len(biden_df)}")

In [ ]:
print(trump_df['tweet'][:10].to_numpy())
print(biden_df['tweet'][:10].to_numpy())

# Post processing files generation

In [ ]:
trump_file_path = f"{GOOGLE_DRIVE_BASE_DIR}/trump_tweets.txt"

In [ ]:
trump_tweets = trump_df["tweet"].tolist()
with open(trump_file_path, "w", encoding="utf-8") as file:
    for tweet in trump_tweets:
        file.write(f"{tweet}\n")

In [ ]:
biden_file_path = f"{GOOGLE_DRIVE_BASE_DIR}/biden_tweets.txt"

In [ ]:
biden_tweets = biden_df["tweet"].tolist()
with open(biden_file_path, "w", encoding="utf-8") as file:
    for tweet in biden_tweets:
        file.write(f"{tweet}\n")

In [ ]:
users_file_path = f"{GOOGLE_DRIVE_BASE_DIR}/users.txt"

In [ ]:
with open(users_file_path, "w", encoding="utf-8") as file:
    for _, tweet in biden_df.iterrows():
        file.write(f"username: {tweet['user_screen_name']}\ndescription: {tweet['user_description']}\n")
    for _, tweet in trump_df.iterrows():
        file.write(f"username: {tweet['user_screen_name']}\ndescription: {tweet['user_description']}\n")